# Calchylus - Lambda calculus in Hy
<br/>
<img src="lambda.png" height=150 width=150 />
<br/>

Calchylus is a combination of [Hy](http://docs.hylang.org) macro and Lambda term dictionary. It was written to help to write and evaluate Lambda expressions.

<blockquote>[Lambda calculus](https://en.wikipedia.org/wiki/Lambda_calculus) is a formal system in mathematical logic for expressing computation based on function abstraction and application using variable binding and substitution. -<i> wikipedia.org</i></blockquote>

The short `𝜆` macro presented below, makes it possible to define and evaluate Lambda expressions the most conventional way. There is really not much to implement because Hy is already a LISP language. LISP, on the other hand, can be defined as *an untyped Lambda calculus extended with constants*. So actually we just need to introduce `𝜆` macro,  simplify the usual LISP notation, and act only with functions.

### Repository

[Calchylus](http://nbviewer.jupyter.org/github/markomanninen/calchylus/blob/master/Calchylus%20-%20Lambda%20calculus%20in%20Hy.ipynb) Jupyter notebook and GitHub [repository](https://github.com/markomanninen/calchylus) was initialized by [Marko Manninen](https://github.com/markomanninen), 08/2017.

##  Lambda (`𝜆`) macro

In [1]:
; index-find function needs to be available at compile time for the lambda expression macro
(eval-and-compile
  ; set comma constant for separator
  ; at the moment Hy doesn't support dot on macro expressions because dot is mixed with internal HyCons functionality
  ; causing this error: https://github.com/hylang/hy/blob/e8ffd412028232cc2cc4fe4bfb10f21ce8ab2565/hy/compiler.py#L2478
  (setv comma '·)
  ; find index of the element from the list. if the element is not found, return -1
  (defn index-find [elm lst]
    (try (.index lst elm) (except [ValueError] -1))))
; lambda expression macro
(defmacro 𝜆 [&rest expr]
  (setv idx (index-find comma expr))
  `(fn ~(cut expr 0 (if (pos? idx) idx 0)) ~@(cut expr (inc idx))))

<function _hy_anon_fn_2 at 0x000001991350E598>

### Basics

#### Syntax

Parentheses `(` `)` to elaborate function scope, group terms, prevent disambiguity, and wrap application. Lambda symbol `𝜆` to mark the beginning of the function. Dot separator `·` to distinct function arguments and function body from each other. 

#### Terms

1. Lambda variables. Any symbol following Hy definitions are accepted. For example single character variable names like `x` and `y`, multi character names `args`, also with capitals `FUNC`, are allowed.

2. Lambda abstraction. For example `(𝜆 x · x)` creates a definition of the function `(fn [x] x)`, or $x \mapsto
x$ in mathematic notation.
3. Lambda application. For example `((𝜆 x · x) 1)` calls for anonymous function with input value `1`.

#### Lambda abstraction

Lambda abstraction is a function definition similar to this:

$$\Large(𝜆 \space x · x)$$

We can examine the generated Hy code of the functions with `macroexpand`:

In [2]:
(macroexpand '(𝜆 x · x))

('fn' ['x'] 'x')

#### Lambda application

Lambda application is the procedure to evaluate declared functions with a given input. Applications are similar to this:

$$\Large((𝜆 \space x · x)\space y) \implies y$$

where the first `x` is a name of the argument of the function. The second `x` is the return value of the function. `y` is the input value passed to the function. Later in the document we will use $\implies$ character to denote, if lambda term reduces to simpler terms or if application evaluates to a certain value.

Let us next utilize above application and pass an argument to so called identity function.

#### Identity function

The identity function returns same value that is passed to the function as an argument. Next we run the simple application of the identity function with the input value `1`:

In [3]:
((𝜆 x · x) 1) ; ((𝜆 x · x) 1) ≡ 1

1

Output is as expected, not very exciting one.

#### Function as an argument

Argument passed to the function can also be yet another function. Say, we want to pass the identity function to the identity function instead of the static value, then the appication would look this:

$$\Large((𝜆 \space x · x) \space (𝜆 \space x · x)) \implies (𝜆 \space x · x)$$

The outcome of such application is a function, thus it alone cannot evaluate to any value. Rather we need to treat it as a function part of the application and provide input for it:

In [4]:
(((𝜆 x · x) (𝜆 x · x)) 1)

1

Again output is expected to be `1` because first we pass identity function `a` to the identity function `b` and then we pass the value `1` to the identity function `b`.

#### Self application

One more step toward deeper nested structures is to call function inside a function:

$$\Large((𝜆 \space x \space · ((𝜆 \space x \space · x) \space x)) \space 1) \implies 1$$

In [5]:
((𝜆 x · ((𝜆 x · x) x)) 1)

1

We can see how the result is same in all three examples.

Note also how variable names are same on nested functions. This works because of safe scoping in Hy, but one might understand functionality more clear if different variable names were used like this:

$$\Large(𝜆 \space x \space · ((𝜆 \space y \space · y) \space x))$$

#### Currying

Restriction of the traditional Lambda calculus is that it allows one and only one argument per function. Programmers on most of the modern languages are used to multiary functions. There is a way to imitate multiple argument passing with a nested structure called *currying* in Lambda calculus.

$$\Large(𝜆 \space x \space · (𝜆 \space y \space · (𝜆 \space z \space · x \space y \space z)))$$

Let us make a simple multiplication with two variables for instance:

In [6]:
((((𝜆 x · (𝜆 y · (𝜆 z · (* x y z)))) 2) 2) 2)

8

#### Many arguments

Also multiple arguments can be passed to the lambda function althought the original specification allows one and only one argument to be passed to the Lambda function.

Support for both zero and multiple arguments simpifies the Lambda expression notation.

In [7]:
((𝜆 x y z · (* x y z)) 2 2 2)

8

#### Constants

Whatever is the argument, return 1:

In [8]:
((𝜆 x · 2) 1)

2

#### No arguments?!

In original Lambda Calculus functions must have one and only one argument. Lambda macro in Calchylus is not that restricted. We can omit argument for simplicity:

In [9]:
((𝜆 · 2))

2

#### Apply function

Further abstracting functional properties of the lambda expressions, we can implement function caller (apply func args) with the next statement:

In [10]:
(((𝜆 func · (𝜆 arg · (func arg))) (𝜆 x · x)) 2)

2

In [11]:
(, ((𝜆 x · x) 1) ((𝜆 x · x) 2) ((𝜆 x · x) 3))

(1, 2, 3)

#### Make pair

$$\Large(𝜆 \space x · (𝜆 \space y · (𝜆 \space f · ((f \space x) \space y))))$$

Select first:

In [12]:
((((𝜆 x · (𝜆 y · (𝜆 f · ((f x) y)))) 1) 2) (𝜆 x · (𝜆 y · x)))

1

Select second:

In [13]:
((((𝜆 x · (𝜆 y · (𝜆 f · ((f x) y)))) 1) 2) (𝜆 x · (𝜆 y · y)))

2

#### Twiddle

Swap function argument pair.

In [14]:
; ask for the second but because arguments are swapped, the first one is returned
((((𝜆 x · (𝜆 y · (𝜆 f · ((f y) x)))) 1) 2) (𝜆 x · (𝜆 y · y)))

1

#### Infinite recursion

Produce infinite loop:

In [15]:
(try
 ; we will catch recursion error from this line, otherwise output would be messier
 ((𝜆 x · (x x)) (𝜆 x · (x x)))
 (except (e RecursionError)
   (print "Eternal loop achieved!")))

Eternal loop achieved!


#### Free and bound variables

Lambda abstractions can hold any free variables, but at the application evaluation, none of the functions should have free variables or it will cause variable definition error. For example, this is ok since we are declaring the function, not executing it:

In [16]:
(𝜆 x · (+ x y))

<function <lambda> at 0x00000199133F7048>

But unbound variable y will become a problem on evaluation of the application:

In [17]:
(try 
  ((𝜆 x · (+ x y)) 1)
  (except (e NameError)
    (print e)))

name 'y' is not defined


### Logical operations

#### Boolean TRUE

Boolean true operator is defined by functional means with a pair selector that returns the first item of the given arguments:

$$\Large(((𝜆 \space x · (𝜆 \space y · x)) \space a) \space b) \implies a$$

In [18]:
; select the first
(((𝜆 x · (𝜆 y · x)) 1) 2)

1

#### Boolean FALSE

Boolean false operator is defined with a pair selector that returns the second item of the given arguments:

$$\Large(((𝜆 \space x · (𝜆 \space y · y)) \space a) \space b) \implies b$$

In [19]:
; select the second
(((𝜆 x · (𝜆 y · y)) 1) 2)

2

#### Conditions

IF-THEN-ELSE condition can be formed with boolean true / false selectors. As we defined earlier, true selector returns the first given argument, and false selector returns the second argument. Thus in Lambda calculus we can produce IF-THEN-ELSE functionality by passing either true or false selector to the function and let it select between the first options and the second option.

Let proposition `P` be the boolean selector, `a` the first option, and `b` the second option. Formal definition of the IF-THEN-ELSE Lambda term would be:

$$\Large(𝜆 \space P · (𝜆 \space a · (𝜆 \space b · ((P \space a) \space b))))$$

In the next example `P` is $(𝜆 \space x · (𝜆 \space y · x))$ (i.e. true), `a` is literal T, and `b` is literal F:

In [20]:
((((𝜆 P · (𝜆 a · (𝜆 b · ((P a) b)))) (𝜆 x · (𝜆 y · x))) 'T) 'F)

'T'

Or with a shorter notation, this time `P` is $(𝜆 \space x · (𝜆 \space y · y))$ (i.e. false):

In [21]:
((𝜆 P a b · (P a b)) (𝜆 x y · y) 'T 'F)

'F'

### Connectives

#### NOT

$$\Large(𝜆 x · ((x (𝜆 x · (𝜆 y · y))) (𝜆 x · (𝜆 y · x))))$$

In [22]:
((((𝜆 x · ((x (𝜆 x · (𝜆 y · y))) (𝜆 x · (𝜆 y · x)))) (𝜆 x · (𝜆 y · x))) 'T) 'F)

'F'

In [23]:
(((𝜆 x · (x (𝜆 x y · y) (𝜆 x y · x))) (𝜆 x y · y)) 'T 'F)

'T'

#### AND

$$\Large(𝜆 a · (𝜆 b · ((a b) (𝜆 x · (𝜆 y · y)))))$$

In [24]:
; IF a is TRUE AND b is TRUE, THEN x, ELSE y
; a is TRUE, b is TRUE, x is T, y is F -> T
(((((𝜆 a · (𝜆 b · ((a b) (𝜆 x · (𝜆 y · y))))) (𝜆 x · (𝜆 y · x))) (𝜆 x · (𝜆 y · x))) 'T) 'F)

'T'

In [25]:
(((𝜆 a b · (a b (𝜆 x y · y))) (𝜆 x y · x) (𝜆 x y · y)) 'T 'F)

'F'

#### OR

$$\Large(𝜆 a · (𝜆 b · ((a (𝜆 x · (𝜆 y · x))) b)))$$

In [26]:
; IF a is TRUE OR b is TRUE, THEN x, ELSE y
; a is TRUE, b is FALSE, x is T, y is F -> T
(((((𝜆 a · (𝜆 b · ((a (𝜆 x · (𝜆 y · x))) b))) (𝜆 x · (𝜆 y · x))) (𝜆 x · (𝜆 y · y))) 'T) 'F)

'T'

In [27]:
(((𝜆 a b · (a (𝜆 x y · x) b)) (𝜆 x y · y) (𝜆 x y · y)) 'T 'F)

'F'

### Y combinator

#### Triangular number

Summation...

In [28]:
; loop until given limit and sum up all values -> (summa 10)
; x = function, y = steps, z = result
((𝜆 x y · (x x y 0))
  (𝜆 x y z · (if (> y 0) (x x (- y 1) (+ y z)) z))
  10)

55

#### Product

In [29]:
; loop until given limit and sum up all values -> (product 7)
; x = function, y = steps, z = result
((𝜆 x y · (x x y 1)) (𝜆 x y z · (if (> y 0) (x x (- y 1) (* y z)) z)) 7)

5040

### Lists

## The [MIT](http://choosealicense.com/licenses/mit/) License

Copyright (c) 2017 Marko Manninen